In [216]:
import random

import nltk
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from globals import create_tables, session_factory
from models.Question import Question
from src.normalizer import normalize_str

In [217]:
create_tables()

In [218]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


False

In [219]:
QUESTION_TO_SEARCH = "Часто я бросаю даже важную деятельность, если усилия, которые требуются, становятся слишком неприятными."
normalized_question = normalize_str(QUESTION_TO_SEARCH, MorphAnalyzer())

In [220]:
with session_factory() as session:
    question_pair = [(QUESTION_TO_SEARCH, normalized_question)] + session.query(Question.raw, Question.transformed).all()
df = pd.DataFrame(question_pair, columns=['raw', 'transformed'])
df.head(10)

,raw,transformed
0,"Часто я бросаю даже важную деятельность, если ...",часто бросать важный деятельность усилие котор...
1,У Вас есть тяга к рисованию?,тяга рисование
2,"Чтобы понять что-либо, Вы часто обращаетесь к ...",понять либо часто обращаться образный представ...
3,"Вам вполне понятен язык знаков, формул, графиков?",вполне понятный язык знак формула график
4,Доставляет ли Вам удовольствие пересказывать у...,доставлять удовольствие пересказывать увидеть ...
5,Вы стараетесь скорее обобщить имеющуюся информ...,стараться скорее обобщить иметься информация у...
6,У Вас обычно много времени занимает переосмысл...,обычно время занимать переосмысление видеть сл...
7,"Вам проще понять что-либо, повертев в своих ру...",простой понять либо повертеть свой рука собрат...
8,Вы стараетесь обогатить свою речь метафорическ...,стараться обогатить речь метафорический выражение
9,"Являются ли рисование, лепка, декоративно-прик...",являться рисование лепка декоративный прикладн...


# Helpers

In [221]:
def print_closest(model_df: pd.DataFrame):
    cos_sim = cosine_similarity(model_df)
    enum_cos_sim = enumerate(cos_sim[0])
    enum_sort_cos_sim = sorted(enum_cos_sim, key=lambda x: x[1], reverse=True)
    print(QUESTION_TO_SEARCH, end="\n\n")
    for i, score in enum_sort_cos_sim[1: 10]:
        print(score, df["raw"][i], sep=": ")

# Tf-Idf

In [222]:
tfidf_vect = TfidfVectorizer(
    analyzer='word',
    max_df=0.95,
    min_df=3,
    ngram_range=(1, 2),
    max_features=100_000,
    stop_words=nltk.corpus.stopwords.words('russian')
)
tfidf_model = pd.DataFrame.sparse.from_spmatrix(
    tfidf_vect.fit_transform(df['transformed'])
)

In [223]:
print_closest(tfidf_model)

Часто я бросаю даже важную деятельность, если усилия, которые требуются, становятся слишком неприятными.

0.4594915400764113: Я слишком раздражителен.
0.3545734229604015: Ваши знакомые говорят Вам, что Вы слишком часто прибегаете к абстракциям?
0.3356153949848156: Как часто вы злитесь по поводу вещей, которые вы не можете контролировать?
0.32321698272450977: Я стараюсь придерживаться диеты, которую сам разработал.
0.3220484889209827: Как часто вы в силах контролировать раздражение?
0.31431192543810343: Для меня важен не один друг, а дружный хороший коллектив.
0.3138746955185958: Я слишком мнителен, без конца тревожусь и беспокоюсь обо всем.
0.2787355381304951: Жизнь научила меня не быть слишком откровенным даже с друзьями.
0.25211067447553254: Люблю одежду модную и необычную, которая невольно привлекает взоры.


# LSA

In [228]:
lsa = TruncatedSVD(
    n_components=100,
    n_iter=1000
)
lsa_model = lsa.fit_transform(tfidf_model)

In [229]:
print_closest(lsa_model)

Часто я бросаю даже важную деятельность, если усилия, которые требуются, становятся слишком неприятными.

0.6277805678639421: Я слишком раздражителен.
0.5278071982023371: Ваши знакомые говорят Вам, что Вы слишком часто прибегаете к абстракциям?
0.5107065245833922: Как часто вы злитесь по поводу вещей, которые вы не можете контролировать?
0.4181867521815881: Я слишком мнителен, без конца тревожусь и беспокоюсь обо всем.
0.41659958109774453: Жизнь научила меня не быть слишком откровенным даже с друзьями.
0.41289837919341627: Я стараюсь придерживаться диеты, которую сам разработал.
0.4090394782710726: Как часто вы в силах контролировать раздражение?
0.359734402341696: Люблю одежду модную и необычную, которая невольно привлекает взоры.
0.33812384986032173: Я ценю такого друга, который умеет меня выслушать, приободрить, вселить уверенность, успокоить.
